In [ ]:
import pandas as pd
import numpy as np
from skimage import color
from skimage.feature import greycomatrix, greycoprops
from PIL import Image
import matplotlib.pyplot as plt



질감 피처로는 Gray-Level Co-occurrence Matrix (GLCM)를 사용할 수 있습니다. GLCM은 이미지 내의 픽셀 간의 밝기 값이 어떻게 함께 나타나는지를 설명하는 통계적 방법으로, 질감 피처를 추출하는데 널리 사용됩니다. GLCM에서는 대표적으로 contrast, dissimilarity, homogeneity, energy, correlation 등의 피처를 계산할 수 있습니다.

In [ ]:
thickness_path = "./data/Thickness.csv"
thickness_df = pd.read_csv(thickness_path)

thickness_df.head()

In [ ]:
img_path_list = []
Top_path = './data/box_img/Top/'
Bot_path = './data/box_img/Bot/'

for i in range(1, 33):
    for j in range(1, 5):
        if i == 9 and j == 1: 
            continue
        img_path_list.append('./data/test_img/AI-{}_0{}_transformed.jpg'.format(i, j))

In [ ]:
top_box = [230:280,500:550]
bottom_box = [700:750,500:550]

In [ ]:
def extract_features(box):
    # Convert the box to grayscale and rescale to 0-255
    gray_box = color.rgb2gray(box)
    gray_box = (gray_box * 255).astype(np.uint8)

    # Compute color features: mean and std of each channel
    mean_rgb = np.mean(box, axis=(0, 1))
    std_rgb = np.std(box, axis=(0, 1))

    # Compute texture features: properties of GLCM
    glcm = greycomatrix(gray_box, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = greycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = greycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = greycoprops(glcm, 'homogeneity')[0, 0]
    energy = greycoprops(glcm, 'energy')[0, 0]
    correlation = greycoprops(glcm, 'correlation')[0, 0]

    return [mean_rgb, std_rgb, contrast, dissimilarity, homogeneity, energy, correlation]

In [ ]:
for i in range(len(img_path_list)):
    img_path = img_path_list[i]
    img = Image.open(img_path)
    img = np.array(img)
    
    top_box = img[230:280,500:550]
    bottom_box = img[700:750,500:550]
    
    top_box = Image.fromarray(top_box)
    bottom_box = Image.fromarray(bottom_box)
    
    top_box.save(Top_path + 'Top_{}.jpg'.format(i))
    bottom_box.save(Bot_path + 'Bot_{}.jpg'.format(i))
    
    # Extract features
    top_features = extract_features(top_box)
    bottom_features = extract_features(bottom_box)

    # Combine into a data frame
    features_df = pd.DataFrame([top_features, bottom_features], columns=['Mean RGB', 'Std RGB', 'Contrast', 'Dissimilarity', 'Homogeneity', 'Energy', 'Correlation'])

features_df